<a href="https://colab.research.google.com/github/marinwitherspoon/Image_Description_Application/blob/main/User_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import packages
import torch
from transformers import AutoModelForCausalLM
from PIL import Image
import numpy as np
import os
from transformers import AutoProcessor, AutoModelForCausalLM

In [2]:
import requests

processor = AutoProcessor.from_pretrained("microsoft/git-base")
loaded_model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")
loaded_optimizer = torch.optim.AdamW(loaded_model.parameters(), lr=5e-5)

def download_file(url, destination):
    response = requests.get(url)
    with open(destination, 'wb') as f:
        f.write(response.content)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/707M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [6]:
# Download and load the model state
model_url = 'https://drive.google.com/file/d/1_fRVYBHAeC7HS76b9OFqiVS9mIWUwUYj/view?usp=drive_link'
model_destination = '/tmp/finetuned_model_grad'
download_file(model_url, model_destination)
loaded_model.load_state_dict(torch.load(model_destination, map_location=torch.device('cpu')))

# Download and load the optimizer state
optimizer_url = 'https://drive.google.com/file/d/13tzP2ti7x3sxAR__GvT2f3PZH-EP2_k6/view?usp=drive_link'
optimizer_destination = '/tmp/optimizer_state_grad'
download_file(optimizer_url, optimizer_destination)
loaded_optimizer.load_state_dict(torch.load(optimizer_destination, map_location=torch.device('cpu')))

# Set the model to evaluation mode for inference
loaded_model.eval()

UnpicklingError: invalid load key, '<'.

In [6]:
# #user input of file path to image
# image_path = input('image file path here: ')
# # example image path: /content/drive/MyDrive/Capstone (Shared)/User input/portrait-photographer-background-people-public-event-56561794.jpg

# # User interface function
# def imageinput(path):
#   # open and print image
#   image = Image.open(path)
#   width, height = image.size
#   display(image.resize((int(0.7*width), int(0.5*height))))

#   device = "cuda" if torch.cuda.is_available() else "cpu"

#   # Run the image through the model
#   inputs = processor(images=image, return_tensors="pt").to(device)
#   pixel_values = inputs.pixel_values
#   generated_ids = loaded_model.generate(pixel_values=pixel_values, max_length=50)
#   #generate and print caption
#   generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
#   print(generated_caption)

# #call function with inputed image path
# imageinput(image_path)

In [7]:
import ipywidgets as widgets
from google.colab import files
from IPython.display import display, Image, clear_output

# File upload button
upload_button = widgets.FileUpload(
    accept='.jpg')

# Clear button
clear_button = widgets.Button(
    description='Clear',
    button_style='danger')

# button display
def b_dis():
    display(widgets.HBox([upload_button, clear_button]))

# Function to handle file upload and display
def on_upload_change(change):
    if change['name'] == 'value':
      clear_output(wait=True)
      b_dis()  # show buttons again after clear
      for filename, file_info in upload_button.value.items():
        display(Image(data=file_info['content'])) # Display image

# Function to clear the uploaded files and output
def on_clear_button_clicked(b):
    upload_button.value.clear()
    clear_output()
    b_dis()  # show buttons again after clear

# call button function
upload_button.observe(on_upload_change, names='value')
clear_button.on_click(on_clear_button_clicked)

# Display the buttons initially
b_dis()


##### UPLOAD TO CODE FILES NOT DRIVE

# website

In [ ]:
!pip install flask
!pip install pyngrok
!ngrok authtoken 2eNEbhmcYSi4IhGNuaFqIGoviaN_2SSszrNLqH12471PcvqHp

In [10]:
from flask import Flask, render_template_string
from pyngrok import ngrok

app = Flask(__name__)

@app.route('/')
def index():
    return render_template_string("""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Interactive Buttons</title>
    </head>
    <body>
        <input type="file" id="file_input">
        <button id="clear_button">Clear</button>
        <img id="uploaded_image" style="display:none;">

        <script>
            document.getElementById('file_input').addEventListener('change', function(event) {
                var reader = new FileReader();
                reader.onload = function(e) {
                    document.getElementById('uploaded_image').src = e.target.result;
                    document.getElementById('uploaded_image').style.display = 'block';
                };
                reader.readAsDataURL(event.target.files[0]);
            });

            document.getElementById('clear_button').addEventListener('click', function() {
                document.getElementById('file_input').value = '';
                document.getElementById('uploaded_image').style.display = 'none';
            });
        </script>
    </body>
    </html>
    """)

if __name__ == '__main__':
    # Set up ngrok tunnel
    ngrok_tunnel = ngrok.connect(5000)
    print('Public URL:', ngrok_tunnel.public_url)
    app.run(port=5000)

# running into problem with opening url

Public URL: https://0b1e-34-145-153-50.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [9]:
#### this secion builds testing site

from IPython.display import HTML

html_code = """
<!DOCTYPE html>
<html>
<head>
    <title>Interactive Buttons</title>
</head>
<body>
    <input type="file" id="file_input">
    <button id="clear_button">Clear</button>
    <img id="uploaded_image" style="display:none;">

    <script>
        document.getElementById('file_input').addEventListener('change', function(event) {
            var reader = new FileReader();
            reader.onload = function(e) {
                document.getElementById('uploaded_image').src = e.target.result;
                document.getElementById('uploaded_image').style.display = 'block';
            };
            reader.readAsDataURL(event.target.files[0]);
        });

        document.getElementById('clear_button').addEventListener('click', function() {
            document.getElementById('file_input').value = '';
            document.getElementById('uploaded_image').style.display = 'none';
        });
    </script>
</body>
</html>
"""

display(HTML(html_code))
